In [1]:
import tomopy
import dxchange
import tomocupy

import numpy as np
import tkinter as tk
import ipywidgets as widgets
import matplotlib.pyplot as plt

from ipywidgets import interact
from types import SimpleNamespace
from tkinter.filedialog import askopenfile

/home/beams/TOMO/conda/miniforge3/envs/tomopyui/lib/python3.9/site-packages/cupyx/jit/_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


In [2]:
# Utility Functions

In [3]:
def read_filename():
    window = tk.Tk()
    window.wm_attributes('-topmost', 1)
    window.withdraw()

    file = askopenfile(mode ='r', filetypes =[('Python Files', '*.h5')])
    window.destroy()
    return file.name

# Load data

In [4]:
# !wget -nc https://anl.box.com/shared/static/or8vlzdu07d8zwxvk50ihwghq39ide0o.npz
# file_name = read_filename()
# fname = '/data/2022-12/Luxi_173.h5'
fname = '/home/beams/TOMO/tmpfdc/Luxi_173.h5'
# dark_file_name = file_name
# flat_file_name = file_name

# Read data, dark and flat fields, projection angles, metadata

In [5]:
# data = np.random.rand(100,256,256) #3d-array with 100 frames 256x256
data, flat, dark, theta = dxchange.read_aps_tomoscan_hdf5(fname)#, sino=(100, 400))
tt, meta_dict = dxchange.read_hdf_meta(fname)
# print(meta_dict)

In [6]:
data.shape 

(1800, 852, 2800)

# Plot projections

In [7]:
def plot_projections(angle=1):
    plt.imshow(data[angle,:],cmap='gray')
interact(plot_projections, angle = widgets.IntSlider(name='Projection', value=data.shape[0]//2,
                                               min=0,
                                               max=data.shape[0]-1,
                                               step=1))

interactive(children=(IntSlider(value=900, description='angle', max=1799), Output()), _dom_classes=('widget-in…

<function __main__.plot_projections(angle=1)>

In [8]:
params_dict = {}
for section in tomocupy.config.RECON_STEPS_PARAMS:
    for key, value in tomocupy.config.SECTIONS[section].items():
        key = key.replace('-', '_')
        params_dict[key] = value['default']

args = SimpleNamespace(**params_dict)

args.reconstruction_type          = 'try'
args.file_name                    = fname
args.rotation_axis_auto           = 'auto'
args.rotation_axis_method         = 'sift' 
args.dtype                        = 'float16'
# args.out_path_name                = '/data/tmpfdc' 
args.out_path_name                = '/home/beams/TOMO/tmpfdc/recs/' 
args.clear_folder                 = True
args.fbp_filter                   = 'shepp' 
args.retrieve_phase_method        = None 
args.remove_stripe_method         = 'vo'
args.pixel_size                   = meta_dict['measurement_instrument_detection_system_objective_resolution'][0] * 1e-4
args.propagation_distance         = meta_dict['measurement_instrument_detector_motor_stack_setup_z'][0]
args.energy                       = meta_dict['measurement_instrument_monochromator_energy'][0]
args.retrieve_phase_alpha         = 0.0008
args.rotation_axis_sift_threshold = 0.5
args.rotation_axis                = -1
args.minus_log                    = True

In [9]:
clrotthandle = tomocupy.FindCenter(args)
args.rotation_axis = clrotthandle.find_center()*2**args.binning
print(f'set rotaion  axis {args.rotation_axis}')

Crop data to the power of 2 sizes to work with 16bit precision, output size in x dimension 2048


set rotaion  axis 1407.6629638671875


In [10]:
clpthandle = tomocupy.GPURec(args)
clpthandle.recon_try()

Using dark fields from /home/beams/TOMO/tmpfdc/Luxi_173.h5
Using flat fields from /home/beams/TOMO/tmpfdc/Luxi_173.h5
Crop data to the power of 2 sizes to work with 16bit precision, output size in x dimension 2048


queue size 000 |  |████████████████████████████████████████| 100.0% 


In [ ]:
# center_search = dxchange.read_tiff_stack(args.out_path_name+ 'recon_slice0425_center1734.16.tiff', in)